# Import libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools

# Store every image in an array

In [3]:
import os
data_dir = (r'.\fire_dataset')
categories = ['non_fire_images', 'fire_images']
for i in categories:
    path = os.path.join(data_dir, i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))    


# Resize each image to same size for fast processing

In [4]:
img_size = 128
image_array = cv2.resize(img_array, (img_size,img_size))

In [5]:
dno = cv2.imread('./fire_dataset/non_fire_images/non_fire.12.png')
dyes = cv2.imread('./fire_dataset/fire_images/fire.2.png')

# Convert each image to grayscale and append into an array

In [6]:
train_data = []

for i in categories:
    train_path = os.path.join(data_dir,i)
    tag = categories.index(i)
    for img in os.listdir(train_path):
        try:
            image_arr = cv2.imread(os.path.join(train_path , img), cv2.IMREAD_GRAYSCALE)
            new_image_array = cv2.resize(image_arr, (img_size,img_size))
            train_data.append([new_image_array , tag])
        except Exception as e:
            pass

# Split the features and target in to X and y

In [7]:
X = []
y = []
for i,j in train_data:
    X.append(i)
    y.append(j)
X = np.array(X).reshape(-1,img_size,img_size)
print(X.shape)
X = X/255.0  
X = X.reshape(-1,128,128,1)

(998, 128, 128)


# One-Hot encode the target column

In [8]:
from keras.utils.np_utils import to_categorical   

y_enc = to_categorical(y, num_classes = 4)

In [9]:
X_train , X_test, y_train, y_test = train_test_split(X , y_enc , test_size = 0.1, random_state = 42)
X_train , X_val, y_train, y_val = train_test_split(X_train , y_train , test_size = 0.1, random_state = 42)

In [11]:
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping

model = Sequential()


model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (128,128,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.5))

model.add(Dense(4, activation = "softmax"))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

epochs = 10

es = EarlyStopping(
    monitor='val_acc', 
    mode='max',
    patience = 3
)

batch_size = 16
imggen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=0,
        zoom_range = 0,
        width_shift_range=0,  
        height_shift_range=0,  
        horizontal_flip=True,  
        vertical_flip=False)


C:\Users\Sathvik Samineni\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [12]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("models/model_weights.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

imggen.fit(X_train)
history = model.fit_generator(imggen.flow(X_train,y_train,batch_size = batch_size),
                              epochs = epochs, validation_data = (X_val,y_val),
                              steps_per_epoch = X_train.shape[0] // batch_size,
                              callbacks=callbacks_list)

Epoch 1/10


C:\Users\SATHVI~1\AppData\Local\Temp/ipykernel_17956/19206160.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(imggen.flow(X_train,y_train,batch_size = batch_size),


50/50 [==============================] - 39s 759ms/step - loss: 0.6198 - accuracy: 0.7184 - val_loss: 0.4877 - val_accuracy: 0.7556
Epoch 2/10
50/50 [==============================] - 38s 759ms/step - loss: 0.4839 - accuracy: 0.7816 - val_loss: 0.3842 - val_accuracy: 0.8778
Epoch 3/10
50/50 [==============================] - 37s 734ms/step - loss: 0.4000 - accuracy: 0.8169 - val_loss: 0.4626 - val_accuracy: 0.8000
Epoch 4/10
50/50 [==============================] - 37s 734ms/step - loss: 0.3777 - accuracy: 0.8270 - val_loss: 0.3484 - val_accuracy: 0.8556
Epoch 5/10
50/50 [==============================] - 37s 744ms/step - loss: 0.3428 - accuracy: 0.8636 - val_loss: 0.3706 - val_accuracy: 0.8222
Epoch 6/10
50/50 [==============================] - 37s 748ms/step - loss: 0.3368 - accuracy: 0.8624 - val_loss: 0.3446 - val_accuracy: 0.8667
Epoch 7/10
50/50 [==============================] - 37s 740ms/step - loss: 0.2945 - accuracy: 0.8775 - val_loss: 0.3479 - val_accuracy: 0.8444
Epoch 8/10

In [14]:
##### serialize model structure to JSON
model_json = model.to_json()
with open(r".\models\firenet_model.model", "w") as json_file:
    json_file.write(model_json)

In [15]:
os.listdir(r"models")

['.ipynb_checkpoints',
 'firenet_model.model',
 'mask_detector.model',
 'model_weights.h5']